### TFX Components for MLOps

Using TFX components and Interactive Context to develop, train and deploy ML pipeline for a multi-class classification model using the [Covertype Dataset](https://github.com/jarokaz/mlops-labs/blob/master/datasets/covertype/README.md)


- #### <font color='blue'> ExampleGen </font>
- #### <font color='blue'> StatisticsGen </font>
- #### <font color='blue'> SchemaGen </font>
- #### <font color='blue'> ExampleValidator </font>

- #### <font color='blue'> Transform </font>
- #### <font color='blue'> Trainer </font>
- #### <font color='blue'> Tuner </font>

- #### <font color='blue'> Evaluator </font>
- #### <font color='blue'> InfraValidator </font>
- #### <font color='blue'> Pusher </font>

 



In [5]:
# Import libraries
import absl
import os
import tempfile
import time

import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
import tfx

from pprint import pprint
from tensorflow_metadata.proto.v0 import schema_pb2, statistics_pb2, anomalies_pb2
from tensorflow_transform.tf_metadata import schema_utils
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import InfraValidator
from tfx.components import Pusher
from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components import Tuner
from tfx.dsl.components.base import executor_spec
from tfx.components.common_nodes.importer_node import ImporterNode
from tfx.components.trainer import executor as trainer_executor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import evaluator_pb2
from tfx.proto import example_gen_pb2
from tfx.proto import infra_validator_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.proto.evaluator_pb2 import SingleSlicingSpec

from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import HyperParameters
from tfx.types.standard_artifacts import ModelBlessing
from tfx.types.standard_artifacts import InfraBlessing

#### Declare pipeline variables

In [6]:
ARTIFACT_STORE = os.path.join(os.sep, 'home', 'jupyter', 'artifact-store')
SERVING_MODEL_DIR=os.path.join(os.sep, 'home', 'jupyter', 'serving_model')
DATA_ROOT = 'home/jupyter/data'

In [7]:
PIPELINE_NAME = 'tfx-covertype-classifier'
PIPELINE_ROOT = os.path.join(ARTIFACT_STORE, PIPELINE_NAME, time.strftime("%Y%m%d_%H%M%S"))
os.makedirs(PIPELINE_ROOT, exist_ok=True)

context = InteractiveContext(
    pipeline_name=PIPELINE_NAME,
    pipeline_root=PIPELINE_ROOT,
    metadata_connection_config=None)

### <font color='blue'> ExampleGen - Data Ingestion</font>
The `ExampleGen` component ingests data into a TFX pipeline which will be used by other downstream components.
- `CsvExampleGen` is used here to ingest CSV files and emit them as `tf.Example` records.
- The data is configured to create 80-20 `train` and `eval` splits.

In [8]:
output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=4),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1)
    ]))

example_gen = tfx.components.CsvExampleGen(
    input_base=DATA_ROOT,
    output_config=output_config)

In [9]:
context.run(example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.


INFO:absl:Processing input csv data gs://cloud-training/OCBL203/workshop-datasets/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:5611885,xor_checksum:1657792105,sum_checksum:1657792105"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            string_value: "0"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
        ))

### <font color='blue'> StatisticsGen - Statistics Generation </font>

The `StatisticsGen`  component generates data statistics that can be used by other TFX components for schema understanding and configuration

In [11]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])

INFO:absl:Excluding no splits because exclude_splits is not set.


In [12]:
context.run(statistics_gen)

INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to /home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/StatisticsGen/statistics/2/train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to /home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/StatisticsGen/statistics/2/eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 7
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
        ))

#### Visualize statistics

In [13]:
context.show(statistics_gen.outputs['statistics'])

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


### <font color='blue'> SchemaGen - Infering data schema </font>

- TFX components use a description input data - schema. 
- The schema is an instance of `schema.proto`. 
- It specifies data types for feature values, whether a feature has to be present, and other properties. 
- The `SchemaGen` component generates the schema using the statistics for the `train` split. The statistics for other splits are ignored.

In [14]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)

INFO:absl:Excluding no splits because exclude_splits is not set.


In [15]:
context.run(schema_gen)

INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
2023-02-13 20:27:35.255082: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:581] No property is defined for the Type
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to /home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 9
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 9
        name: "Schema"
        )]
        ))

#### Visualize the inferred schema

In [16]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Soil_Type',STRING,required,single,'Soil_Type'
'Wilderness_Area',STRING,required,single,'Wilderness_Area'
'Aspect',INT,required,single,-
'Cover_Type',INT,required,single,-
'Elevation',INT,required,single,-
'Hillshade_3pm',INT,required,single,-
'Hillshade_9am',INT,required,single,-
'Hillshade_Noon',INT,required,single,-
'Horizontal_Distance_To_Fire_Points',INT,required,single,-


/home/jupyter/.local/lib/python3.7/site-packages/tensorflow_data_validation/utils/display_util.py:151: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


#### Modify and update schema

In [18]:
# Load the auto-generated schema proto file
schema_proto_path = '{}/{}'.format(schema_gen.outputs['schema'].get()[0].uri, 'schema.pbtxt')
schema = tfdv.load_schema_text(schema_proto_path)

# Modify the schema
# restrict categorical feature 'Cover_Type' between the values of 0 and 6.
# restrict numeric feature 'Slope' between 0 and 90.
tfdv.set_domain(schema, 'Cover_Type', schema_pb2.IntDomain(name='Cover_Type', min=0, max=6, is_categorical=True))
tfdv.set_domain(schema, 'Slope',  schema_pb2.IntDomain(name='Slope', min=0, max=90))


tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Soil_Type',STRING,required,single,'Soil_Type'
'Wilderness_Area',STRING,required,single,'Wilderness_Area'
'Aspect',INT,required,single,-
'Cover_Type',INT,required,single,"[0,6]"
'Elevation',INT,required,single,-
'Hillshade_3pm',INT,required,single,-
'Hillshade_9am',INT,required,single,-
'Hillshade_Noon',INT,required,single,-
'Horizontal_Distance_To_Fire_Points',INT,required,single,-


/home/jupyter/.local/lib/python3.7/site-packages/tensorflow_data_validation/utils/display_util.py:151: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


#### Save the updated schema 

In [19]:
schema_dir = os.path.join(ARTIFACT_STORE, 'schema')
tf.io.gfile.makedirs(schema_dir)
schema_file = os.path.join(schema_dir, 'schema.pbtxt')

tfdv.write_schema_text(schema, schema_file)

In [20]:
# Import updated schema using ImporterNode
schema_importer = ImporterNode(
    instance_name='Schema_Importer',
    source_uri=schema_dir,
    artifact_type=tfx.types.standard_artifacts.Schema,
    reimport=False)

In [21]:
context.run(schema_importer)

INFO:absl:Running driver for ImporterNode.Schema_Importer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Processing source uri: /home/jupyter/artifact-store/schema, properties: {}, custom_properties: {}
INFO:absl:Running executor for ImporterNode.Schema_Importer
INFO:absl:Running publisher for ImporterNode.Schema_Importer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ImporterNode.Schema_Importer
    execution_id: 4
    outputs:
        result: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 4
        type_id: 9
        uri: "/home/jupyter/artifact-store/schema"
        , artifact_type: id: 9
        name: "Schema"
        )]
        ))

In [22]:
# Visualize  imported schema
context.show(schema_importer.outputs['result'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Soil_Type',STRING,required,single,'Soil_Type'
'Wilderness_Area',STRING,required,single,'Wilderness_Area'
'Aspect',INT,required,single,-
'Cover_Type',INT,required,single,"[0,6]"
'Elevation',INT,required,single,-
'Hillshade_3pm',INT,required,single,-
'Hillshade_9am',INT,required,single,-
'Hillshade_Noon',INT,required,single,-
'Horizontal_Distance_To_Fire_Points',INT,required,single,-


/home/jupyter/.local/lib/python3.7/site-packages/tensorflow_data_validation/utils/display_util.py:151: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


### <font color='blue'> ExampleValidator - Validate Data </font>

The `ExampleValidator` component identifies anomalies in data.

In [23]:
example_validator = ExampleValidator(
    instance_name="Data_Validation",    
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_importer.outputs['result'])


INFO:absl:Excluding no splits because exclude_splits is not set.


In [24]:
context.run(example_validator)

INFO:absl:Running driver for ExampleValidator.Data_Validation
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator.Data_Validation
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to /home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/ExampleValidator.Data_Validation/anomalies/5/train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to /home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/ExampleValidator.Data_Validation/anomalies/5/eval.
INFO:absl:Running publisher for ExampleValidator.Data_Validation
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator.Data_Validation
    execution_id: 5
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 5
        type_id: 12
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/ExampleValidator.Data_Validation/anomalies/5"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator.Data_Validation"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 12
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
        ))

In [26]:
# Examine the output of ExampleValidator
context.show(example_validator.outputs['output'])

### <font color='blue'> Transform - Preprocessing Data </font>

- The `Transform` component performs data transformation and feature engineering. 
- This  can then be used by serving components to make sure that the same data pre-processing logic is applied at training and serving.
- `Trainsform` pre-processing code is configured in `preprocessing_fn` function which is called when the `Transform` component runs.


In [ ]:
FEATURES = 'features.py'

In [28]:
%%writefile {FEATURES}
import tensorflow as tf
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import schema_utils

NUMERIC_FEATURE_KEYS = [
    'Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
    'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
    'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
    'Horizontal_Distance_To_Fire_Points'
]

CATEGORICAL_FEATURE_KEYS = ['Wilderness_Area', 'Soil_Type']

LABEL_KEY = 'Cover_Type'
NUM_CLASSES = 7

def transformed_name(key):
    return key + '_xf'


Overwriting features.py


In [29]:
TRANSFORM_MODULE = 'preprocessing.py'

In [30]:
%%writefile {TRANSFORM_MODULE}
"""Covertype preprocessing - This file defines a template for TFX Transform component."""

import tensorflow as tf
import tensorflow_transform as tft

import features

def _fill_in_missing(x):
    """Replace missing values in a SparseTensor.
    Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
        Args:
            x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
              in the second dimension.
        Returns:
            A rank 1 tensor where missing values of `x` have been filled in.
    """
    default_value = '' if x.dtype == tf.string else 0
    return tf.squeeze(
      tf.sparse.to_dense(
          tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
          default_value),
      axis=1)

def preprocessing_fn(inputs):
    """Preprocesses Covertype Dataset."""

    outputs = {}

    # Scale numerical features.
    for key in features.NUMERIC_FEATURE_KEYS:
        outputs[features.transformed_name(key)] = tft.scale_to_z_score(
            _fill_in_missing(inputs[key]))

    # Generate vocabularies and maps categorical features.
    for key in features.CATEGORICAL_FEATURE_KEYS:
        outputs[features.transformed_name(key)] = tft.compute_and_apply_vocabulary(
                x=_fill_in_missing(inputs[key]), num_oov_buckets=1, vocab_filename=key)

    # Convert Cover_Type to dense tensor.
    outputs[features.transformed_name(features.LABEL_KEY)] = _fill_in_missing(
      inputs[features.LABEL_KEY])

    return outputs


Overwriting preprocessing.py


In [31]:
# Configure and run the Transform component.
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_importer.outputs['result'],
    module_file=TRANSFORM_MODULE)


context.run(transform)

INFO:absl:Running driver for Transform
INFO:absl:MetadataStore with DB connection initialized
2023-02-13 20:31:45.849702: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:581] No property is defined for the Type
2023-02-13 20:31:45.856089: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:581] No property is defined for the Type
INFO:absl:Running executor for Transform
INFO:absl:Analyze the 'train' split and transform all splits when splits_config is not set.


Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`


Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`
INFO:absl:Feature Soil_Type has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Wilderness_Area has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Aspect has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Cover_Type has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Elevation has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Hillshade_3pm has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Hillshade_9am has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Hillshade_Noon has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Horizontal_Distance_To_Fire_Points has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Horizontal_Distance_To_Hydrology has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Horizontal_Distance_To_Roadways has no shape. Setting to VarLenSparseTens

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.
INFO:absl:Feature Soil_Type has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Wilderness_Area has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Aspect has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Cover_Type has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Elevation has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Hillshade_3pm has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Hillshade_9am has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Hillshade_Noon has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Horizontal_Distance_To_Fire_Points has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Horizontal_Distance_To_Hydrology has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Horizontal_Distance_To_Roadways has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Slope has no shape. Setting to VarLenSparseTen

...

value: "\n\013\n\tConst_5:0\022\tSoil_Type"



value: "\n\013\n\tConst_5:0\022\tSoil_Type"



INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 6
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 6
        type_id: 14
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/Transform/transform_graph/6"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "TransformGraph"
        )]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 7
        type_id: 5
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/Transform/transformed_examples/6"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 8
        type_id: 15
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/Transform/updated_analyzer_cache/6"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 15
        name: "TransformCache"
        )]
        ))

#### Examine the `Transform` component's outputs

The Transform component has 2 outputs:

- `transform_graph` - contains the graph that can perform the preprocessing operations (this graph will be included in the serving and evaluation models).
- `transformed_examples` - contains the preprocessed training and evaluation data.

In [32]:
# a peek of `transform_graph` artifact: points to a dir containing 3 subdirectories:
os.listdir(transform.outputs['transform_graph'].get()[0].uri)

['metadata', 'transform_fn', 'transformed_metadata']

In [33]:
# a peek of `transform.examples` artifact
os.listdir(transform.outputs['transformed_examples'].get()[0].uri)

['train', 'eval']

### <font color='blue'> Trainer - Trains TensorFlow model </font>

- The `Trainer` component trains a model using TensorFlow.
- To configure `Trainer`, you need to encapsulate your training code in a Python module that is then provided to the `Trainer` as an input. 


In [9]:
TRAINER_MODULE_FILE = 'model.py'

In [10]:
%%writefile {TRAINER_MODULE_FILE}
"""Covertype Keras WideDeep Classifier."""

import functools
import absl
import os
from typing import List, Text

import kerastuner
import tensorflow as tf
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import schema_utils

from tfx.components.trainer.executor import TrainerFnArgs
from tfx.components.trainer.fn_args_utils import DataAccessor
from tfx.components.tuner.component import TunerFnResult
from tfx_bsl.tfxio import dataset_options

import features

EPOCHS = 1
TRAIN_BATCH_SIZE = 64
EVAL_BATCH_SIZE = 64


def _gzip_reader_fn(filenames):
    """Small utility returning a record reader that can read gzip'ed files."""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')


def _get_serve_tf_examples_fn(model, tf_transform_output):
    """Returns a function that parses a serialized tf.Example and applies TFT."""

    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        """Returns the output to be used in the serving signature."""
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(features.LABEL_KEY)
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

        transformed_features = model.tft_layer(parsed_features)

        return model(transformed_features)

    return serve_tf_examples_fn



def _input_fn(file_pattern: List[Text],
              data_accessor: DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
    """Generates features and label for tuning/training.

        Args:
            file_pattern: List of paths or patterns of input tfrecord files.
            data_accessor: DataAccessor for converting input to RecordBatch.
            tf_transform_output: A TFTransformOutput.
            batch_size: representing the number of consecutive elements of returned
                  dataset to combine in a single batch

        Returns:
            A dataset that contains (features, indices) tuple where features is a
              dictionary of Tensors, and indices is a single Tensor of label indices.
    """
    dataset = data_accessor.tf_dataset_factory(
          file_pattern,
          dataset_options.TensorFlowDatasetOptions(
              batch_size=batch_size, label_key=features.transformed_name(features.LABEL_KEY)),
          tf_transform_output.transformed_metadata.schema)
    
    return dataset



def _get_hyperparameters() -> kerastuner.HyperParameters:
    """Returns hyperparameters for building Keras model."""
    hp = kerastuner.HyperParameters()
    # Defines search space.
    hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4], default=1e-3)
    hp.Int('n_layers', 1, 2, default=1)
    with hp.conditional_scope('n_layers', 1):
        hp.Int('n_units_1', min_value=8, max_value=128, step=8, default=8)
    with hp.conditional_scope('n_layers', 2):
        hp.Int('n_units_1', min_value=8, max_value=128, step=8, default=8)
        hp.Int('n_units_2', min_value=8, max_value=128, step=8, default=8)        

    return hp


def _build_keras_model(hparams: kerastuner.HyperParameters, 
                       tf_transform_output: tft.TFTransformOutput) -> tf.keras.Model:
    """Creates a Keras WideDeep Classifier model.
        Args:
            hparams: Holds HyperParameters for tuning.
            tf_transform_output: A TFTransformOutput.
        Returns:
            A keras Model.
    """
    deep_columns = [
          tf.feature_column.numeric_column(
              key=features.transformed_name(key), 
              shape=())
          for key in features.NUMERIC_FEATURE_KEYS
    ]
    
    input_layers = {
      column.key: tf.keras.layers.Input(name=column.key, shape=(), dtype=tf.float32)
        for column in deep_columns
  }    

    categorical_columns = [
      tf.feature_column.categorical_column_with_identity(
          key=features.transformed_name(key), 
          num_buckets=tf_transform_output.num_buckets_for_transformed_feature(features.transformed_name(key)), 
          default_value=0)
        for key in features.CATEGORICAL_FEATURE_KEYS
  ]

    wide_columns = [
      tf.feature_column.indicator_column(categorical_column)
        for categorical_column in categorical_columns
  ]
    
    input_layers.update({
      column.categorical_column.key: tf.keras.layers.Input(name=column.categorical_column.key, shape=(), dtype=tf.int32)
        for column in wide_columns
  })


    deep = tf.keras.layers.DenseFeatures(deep_columns)(input_layers)
    for n in range(int(hparams.get('n_layers'))):
        deep = tf.keras.layers.Dense(units=hparams.get('n_units_' + str(n + 1)))(deep)

    wide = tf.keras.layers.DenseFeatures(wide_columns)(input_layers)

    output = tf.keras.layers.Dense(features.NUM_CLASSES, activation='softmax')(
               tf.keras.layers.concatenate([deep, wide]))

    model = tf.keras.Model(input_layers, output)
    model.compile(
      loss='sparse_categorical_crossentropy',
      optimizer=tf.keras.optimizers.Adam(lr=hparams.get('learning_rate')),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    
    model.summary(print_fn=absl.logging.info)

    return model    


# TFX Tuner will call this function.
def tuner_fn(fn_args: TrainerFnArgs) -> TunerFnResult:
    """Build the tuner using the KerasTuner API.
    Args:
        fn_args: Holds args as name/value pairs.
            - working_dir: working dir for tuning.
            - train_files: List of file paths containing training tf.Example data.
            - eval_files: List of file paths containing eval tf.Example data.
            - train_steps: number of train steps.
            - eval_steps: number of eval steps.
            - schema_path: optional schema of the input data.
            - transform_graph_path: optional transform graph produced by TFT.
    Returns:
        A namedtuple contains the following:
        - tuner: A BaseTuner that will be used for tuning.
        - fit_kwargs: Args to pass to tuner's run_trial function for fitting the
                    model , e.g., the training and validation dataset. Required
                    args depend on the above tuner's implementation.
    """
    
    transform_graph = tft.TFTransformOutput(fn_args.transform_graph_path)

    # Construct a build_keras_model_fn that just takes hyperparams from get_hyperparameters as input.
    build_keras_model_fn = functools.partial(
      _build_keras_model, tf_transform_output=transform_graph)  

    # BayesianOptimization is a subclass of kerastuner.Tuner which inherits from BaseTuner.    
    tuner = kerastuner.BayesianOptimization(
      build_keras_model_fn,
      max_trials=10,
      hyperparameters=_get_hyperparameters(),
      objective=kerastuner.Objective('val_sparse_categorical_accuracy', 'max'),
      directory=fn_args.working_dir,
      project_name='covertype_tuning')
  
    train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      transform_graph,
      batch_size=TRAIN_BATCH_SIZE)

    eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      transform_graph,
      batch_size=EVAL_BATCH_SIZE)

    return TunerFnResult(
      tuner=tuner,
      fit_kwargs={
          'x': train_dataset,
          'validation_data': eval_dataset,
          'steps_per_epoch': fn_args.train_steps,
          'validation_steps': fn_args.eval_steps
      })


# TFX Trainer will call this function.
def run_fn(fn_args: TrainerFnArgs):
    """Train the model based on given args.
    Args:
        fn_args: Holds args used to train the model as name/value pairs.
    """

    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

    train_dataset = _input_fn(
      fn_args.train_files, 
      fn_args.data_accessor, 
      tf_transform_output, 
      TRAIN_BATCH_SIZE)

    eval_dataset = _input_fn(
      fn_args.eval_files, 
      fn_args.data_accessor,
      tf_transform_output, 
      EVAL_BATCH_SIZE)

    if fn_args.hyperparameters:
        hparams = kerastuner.HyperParameters.from_config(fn_args.hyperparameters)
    else:
        hparams = _get_hyperparameters()
    
    absl.logging.info('HyperParameters for training: %s' % hparams.get_config())
  

    # Distribute training over multiple replicas on the same machine.
    mirrored_strategy = tf.distribute.MirroredStrategy()
    with mirrored_strategy.scope():
        model = _build_keras_model(
            hparams=hparams,
            tf_transform_output=tf_transform_output)

        
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=fn_args.model_run_dir, update_freq='batch')

    model.fit(
      train_dataset,
      epochs=EPOCHS,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps,
      callbacks=[tensorboard_callback])
    
    signatures = {
      'serving_default':
          _get_serve_tf_examples_fn(model,
                                    tf_transform_output).get_concrete_function(
                                        tf.TensorSpec(
                                            shape=[None],
                                            dtype=tf.string,
                                            name='examples')),
    }
  

    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)
    

Overwriting model.py


#### Create and run the Trainer component
- The `Trainer` component only supports passing a single field - `num_steps` - through the `train_args` and `eval_args` arguments. 

In [40]:
trainer = Trainer(
    custom_executor_spec=executor_spec.ExecutorClassSpec(trainer_executor.GenericExecutor),
    module_file=TRAINER_MODULE_FILE,
    transformed_examples=transform.outputs.transformed_examples,
    schema=schema_importer.outputs.result,
    transform_graph=transform.outputs.transform_graph,
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=5000),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=1000))

In [41]:
context.run(trainer)

INFO:absl:Running driver for Trainer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Trainer
INFO:absl:Training model.
INFO:absl:Feature Aspect_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Cover_Type_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Elevation_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Hillshade_3pm_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Hillshade_9am_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Hillshade_Noon_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Horizontal_Distance_To_Fire_Points_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Horizontal_Distance_To_Hydrology_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Horizontal_Distance_To_Roadways_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Slope_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Soil_Type_xf has a shape . Setting to DenseTensor.
INFO:ab

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
INFO:absl:Model: "functional_3"
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Layer (type)                    Output Shape         Param #     Connected to                     
INFO:absl:==================================================================================================
INFO:absl:Aspect_xf (InputLayer)          [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Elevation_xf (InputLayer)       [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Hillshade_3pm_xf (InputLayer)   [(None,)]            0                                       

   2/5000 [..............................] - ETA: 0s - loss: 2.0972 - sparse_categorical_accuracy: 0.031 - ETA: 2:52 - loss: 2.0837 - sparse_categorical_accuracy: 0.0234WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0029s vs `on_train_batch_end` time: 0.0681s). Check your callbacks.


2023-02-13 20:33:35.998530: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2023-02-13 20:33:36.057345: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: /home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/Trainer/model_run/8/train/plugins/profile/2023_02_13_20_33_36
2023-02-13 20:33:36.059819: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to /home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/Trainer/model_run/8/train/plugins/profile/2023_02_13_20_33_36/tensorflow-2-3-20230213-162057.trace.json.gz
2023-02-13 20:33:36.061708: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: /home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/Trainer/model_run/8/train/plugins/profile/2023_02_13_20_33_36
2023-02-13 20:33:36.061826: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped 

5000/5000 [==============================] - ETA: 24s - loss: 2.0745 - sparse_categorical_accuracy: 0.110 - ETA: 16s - loss: 2.0024 - sparse_categorical_accuracy: 0.16 - ETA: 14s - loss: 1.9402 - sparse_categorical_accuracy: 0.21 - ETA: 13s - loss: 1.8793 - sparse_categorical_accuracy: 0.26 - ETA: 12s - loss: 1.8176 - sparse_categorical_accuracy: 0.30 - ETA: 11s - loss: 1.7622 - sparse_categorical_accuracy: 0.33 - ETA: 11s - loss: 1.7085 - sparse_categorical_accuracy: 0.36 - ETA: 11s - loss: 1.6569 - sparse_categorical_accuracy: 0.38 - ETA: 10s - loss: 1.6106 - sparse_categorical_accuracy: 0.40 - ETA: 10s - loss: 1.5667 - sparse_categorical_accuracy: 0.42 - ETA: 10s - loss: 1.5268 - sparse_categorical_accuracy: 0.43 - ETA: 10s - loss: 1.4902 - sparse_categorical_accuracy: 0.45 - ETA: 10s - loss: 1.4627 - sparse_categorical_accuracy: 0.46 - ETA: 10s - loss: 1.4504 - sparse_categorical_accuracy: 0.46 - ETA: 10s - loss: 1.4234 - sparse_categorical_accuracy: 0.47 - ETA: 10s - loss: 1.4062 

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Assets written to: /home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/Trainer/model/8/serving_model_dir/assets


INFO:tensorflow:Assets written to: /home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/Trainer/model/8/serving_model_dir/assets
INFO:absl:Training complete. Model written to /home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/Trainer/model/8/serving_model_dir. ModelRun written to /home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/Trainer/model_run/8
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 8
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 11
        type_id: 17
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/Trainer/model/8"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 17
        name: "Model"
        )]
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 12
        type_id: 18
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/Trainer/model_run/8"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "ModelRun"
        )]
        ))

### <font color='blue'> Tuner - Tune model's hyperparameters </font>

- The `Tuner` component makes use of the Python [KerasTuner](https://keras-team.github.io/keras-tuner/) API to tune model's hyperparameters. 
- It tighty integrates with the `Transform` and `Trainer` components for model hyperparameter tuning in continuous training pipelines.
- `Tuner` tunes the hyperparameters and emits the best results that can be directly fed into the `Trainer` component during model re-training.

In [43]:
tuner = Tuner(
        module_file=TRAINER_MODULE_FILE,
        examples=transform.outputs['transformed_examples'],
        transform_graph=transform.outputs['transform_graph'],
        train_args=trainer_pb2.TrainArgs(num_steps=1000),
        eval_args=trainer_pb2.EvalArgs(num_steps=500))

In [44]:
context.run(tuner)

INFO:absl:Running driver for Tuner
INFO:absl:MetadataStore with DB connection initialized
2023-02-13 20:33:56.539303: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:581] No property is defined for the Type
INFO:absl:Running executor for Tuner
INFO:absl:Creating temp directory at /home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/.temp/9/
INFO:absl:Train on the 'train' split when train_args.splits is not set.
INFO:absl:Evaluate on the 'eval' split when eval_args.splits is not set.
INFO:absl:Model: "functional_1"
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Layer (type)                    Output Shape         Param #     Connected to                     
INFO:absl:==================================================================================================
INFO:absl:Aspect_xf (InputLayer)          [(None,)]            0                                            
INFO:absl:____

INFO:absl:Start tuning... Tuner ID: tuner0
INFO:absl:Model: "functional_1"
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Layer (type)                    Output Shape         Param #     Connected to                     
INFO:absl:==================================================================================================
INFO:absl:Aspect_xf (InputLayer)          [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Elevation_xf (InputLayer)       [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Hillshade_3pm_xf (InputLayer)   [(None,)]            0                                            
INFO:absl:___________________________________________

1000/1000 [==============================] - ETA: 0s - loss: 2.0400 - sparse_categorical_accuracy: 0.156 - ETA: 1s - loss: 1.6216 - sparse_categorical_accuracy: 0.451 - ETA: 1s - loss: 1.4079 - sparse_categorical_accuracy: 0.535 - ETA: 1s - loss: 1.2670 - sparse_categorical_accuracy: 0.577 - ETA: 1s - loss: 1.1649 - sparse_categorical_accuracy: 0.602 - ETA: 1s - loss: 1.0952 - sparse_categorical_accuracy: 0.619 - ETA: 1s - loss: 1.0413 - sparse_categorical_accuracy: 0.631 - ETA: 1s - loss: 1.0031 - sparse_categorical_accuracy: 0.638 - ETA: 1s - loss: 0.9686 - sparse_categorical_accuracy: 0.646 - ETA: 1s - loss: 0.9429 - sparse_categorical_accuracy: 0.650 - ETA: 1s - loss: 0.9252 - sparse_categorical_accuracy: 0.652 - ETA: 1s - loss: 0.9083 - sparse_categorical_accuracy: 0.656 - ETA: 1s - loss: 0.8930 - sparse_categorical_accuracy: 0.659 - ETA: 1s - loss: 0.8814 - sparse_categorical_accuracy: 0.659 - ETA: 1s - loss: 0.8689 - sparse_categorical_accuracy: 0.661 - ETA: 1s - loss: 0.8589 - 

INFO:absl:Model: "functional_1"
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Layer (type)                    Output Shape         Param #     Connected to                     
INFO:absl:==================================================================================================
INFO:absl:Aspect_xf (InputLayer)          [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Elevation_xf (InputLayer)       [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Hillshade_3pm_xf (InputLayer)   [(None,)]            0                                            
INFO:absl:______________________________________________________________________________________

1000/1000 [==============================] - ETA: 0s - loss: 2.0337 - sparse_categorical_accuracy: 0.171 - ETA: 2s - loss: 1.2924 - sparse_categorical_accuracy: 0.527 - ETA: 2s - loss: 1.0591 - sparse_categorical_accuracy: 0.594 - ETA: 2s - loss: 0.9628 - sparse_categorical_accuracy: 0.621 - ETA: 2s - loss: 0.9041 - sparse_categorical_accuracy: 0.638 - ETA: 2s - loss: 0.8691 - sparse_categorical_accuracy: 0.646 - ETA: 2s - loss: 0.8404 - sparse_categorical_accuracy: 0.657 - ETA: 1s - loss: 0.8232 - sparse_categorical_accuracy: 0.663 - ETA: 1s - loss: 0.8078 - sparse_categorical_accuracy: 0.666 - ETA: 1s - loss: 0.7957 - sparse_categorical_accuracy: 0.669 - ETA: 1s - loss: 0.7860 - sparse_categorical_accuracy: 0.672 - ETA: 1s - loss: 0.7759 - sparse_categorical_accuracy: 0.677 - ETA: 1s - loss: 0.7646 - sparse_categorical_accuracy: 0.681 - ETA: 1s - loss: 0.7583 - sparse_categorical_accuracy: 0.682 - ETA: 1s - loss: 0.7520 - sparse_categorical_accuracy: 0.684 - ETA: 1s - loss: 0.7470 - 

...

1000/1000 [==============================] - ETA: 0s - loss: 2.1705 - sparse_categorical_accuracy: 0.140 - ETA: 1s - loss: 1.4068 - sparse_categorical_accuracy: 0.534 - ETA: 1s - loss: 1.1511 - sparse_categorical_accuracy: 0.599 - ETA: 1s - loss: 1.0122 - sparse_categorical_accuracy: 0.639 - ETA: 1s - loss: 0.9416 - sparse_categorical_accuracy: 0.651 - ETA: 1s - loss: 0.8963 - sparse_categorical_accuracy: 0.660 - ETA: 1s - loss: 0.8663 - sparse_categorical_accuracy: 0.667 - ETA: 1s - loss: 0.8395 - sparse_categorical_accuracy: 0.673 - ETA: 1s - loss: 0.8210 - sparse_categorical_accuracy: 0.676 - ETA: 1s - loss: 0.8047 - sparse_categorical_accuracy: 0.679 - ETA: 1s - loss: 0.7925 - sparse_categorical_accuracy: 0.681 - ETA: 1s - loss: 0.7836 - sparse_categorical_accuracy: 0.682 - ETA: 0s - loss: 0.7737 - sparse_categorical_accuracy: 0.683 - ETA: 0s - loss: 0.7657 - sparse_categorical_accuracy: 0.685 - ETA: 0s - loss: 0.7586 - sparse_categorical_accuracy: 0.687 - ETA: 0s - loss: 0.7515 - 

INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit
INFO:absl:Finished tuning... Tuner ID: tuner0


INFO:absl:Best HyperParameters: {'space': [{'class_name': 'Choice', 'config': {'name': 'learning_rate', 'default': 0.001, 'values': [0.01, 0.001, 0.0001], 'ordered': True}}, {'class_name': 'Int', 'config': {'name': 'n_layers', 'default': 1, 'min_value': 1, 'max_value': 2, 'step': 1, 'sampling': None}}, {'class_name': 'Int', 'config': {'name': 'n_layers=1/n_units_1', 'default': 8, 'min_value': 8, 'max_value': 128, 'step': 8, 'sampling': None}}, {'class_name': 'Int', 'config': {'name': 'n_layers=2/n_units_1', 'default': 8, 'min_value': 8, 'max_value': 128, 'step': 8, 'sampling': None}}, {'class_name': 'Int', 'config': {'name': 'n_layers=2/n_units_2', 'default': 8, 'min_value': 8, 'max_value': 128, 'step': 8, 'sampling': None}}], 'values': {'learning_rate': 0.01, 'n_layers': 1, 'n_layers=1/n_units_1': 32, 'n_layers=2/n_units_1': 104, 'n_layers=2/n_units_2': 88}}
INFO:absl:Best Hyperparameters are written to /home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/Tuner/best_h

ExecutionResult(
    component_id: Tuner
    execution_id: 9
    outputs:
        best_hyperparameters: Channel(
            type_name: HyperParameters
            artifacts: [Artifact(artifact: id: 13
        type_id: 20
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/Tuner/best_hyperparameters/9"
        custom_properties {
          key: "name"
          value {
            string_value: "best_hyperparameters"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Tuner"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "HyperParameters"
        )]
        ))

#### Retrain your model by running Tuner with the best hyperparameters

In [45]:
hparams_importer = ImporterNode(
    instance_name='import_hparams',
    source_uri=tuner.outputs.best_hyperparameters.get()[0].uri,
    artifact_type=HyperParameters)

In [46]:
context.run(hparams_importer)

INFO:absl:Running driver for ImporterNode.import_hparams
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Processing source uri: /home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/Tuner/best_hyperparameters/9, properties: {}, custom_properties: {}
INFO:absl:Reusing existing artifact
INFO:absl:Running executor for ImporterNode.import_hparams
INFO:absl:Running publisher for ImporterNode.import_hparams
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ImporterNode.import_hparams
    execution_id: 10
    outputs:
        result: Channel(
            type_name: HyperParameters
            artifacts: [Artifact(artifact: id: 13
        type_id: 20
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/Tuner/best_hyperparameters/9"
        custom_properties {
          key: "name"
          value {
            string_value: "best_hyperparameters"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Tuner"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        create_time_since_epoch: 1676320436546
        last_update_time_since_epoch: 1676320482085
        , artifact_type: id: 20
        name: "HyperParameters"
        )]
        ))

In [47]:
trainer = Trainer(
    custom_executor_spec=executor_spec.ExecutorClassSpec(trainer_executor.GenericExecutor),
    module_file=TRAINER_MODULE_FILE,
    transformed_examples=transform.outputs.transformed_examples,
    schema=schema_importer.outputs.result,
    transform_graph=transform.outputs.transform_graph,
    hyperparameters=hparams_importer.outputs.result,    
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=5000),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=1000))

In [48]:
context.run(trainer)

INFO:absl:Running driver for Trainer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Trainer
INFO:absl:Training model.
INFO:absl:Feature Aspect_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Cover_Type_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Elevation_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Hillshade_3pm_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Hillshade_9am_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Hillshade_Noon_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Horizontal_Distance_To_Fire_Points_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Horizontal_Distance_To_Hydrology_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Horizontal_Distance_To_Roadways_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Slope_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Soil_Type_xf has a shape . Setting to DenseTensor.
INFO:ab

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
INFO:absl:Model: "functional_3"
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Layer (type)                    Output Shape         Param #     Connected to                     
INFO:absl:==================================================================================================
INFO:absl:Aspect_xf (InputLayer)          [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Elevation_xf (InputLayer)       [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Hillshade_3pm_xf (InputLayer)   [(None,)]            0                                       

   1/5000 [..............................] - ETA: 0s - loss: 2.1382 - sparse_categorical_accuracy: 0.1875

2023-02-13 20:34:45.436648: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.


...

5000/5000 [==============================] - ETA: 2:05 - loss: 1.3733 - sparse_categorical_accuracy: 0.5404 - ETA: 1:01 - loss: 1.0776 - sparse_categorical_accuracy: 0.617 - ETA: 46s - loss: 0.9889 - sparse_categorical_accuracy: 0.638 - ETA: 36s - loss: 0.9286 - sparse_categorical_accuracy: 0.64 - ETA: 31s - loss: 0.8892 - sparse_categorical_accuracy: 0.65 - ETA: 27s - loss: 0.8556 - sparse_categorical_accuracy: 0.66 - ETA: 24s - loss: 0.8316 - sparse_categorical_accuracy: 0.66 - ETA: 22s - loss: 0.8126 - sparse_categorical_accuracy: 0.67 - ETA: 20s - loss: 0.7937 - sparse_categorical_accuracy: 0.67 - ETA: 18s - loss: 0.7783 - sparse_categorical_accuracy: 0.68 - ETA: 17s - loss: 0.7682 - sparse_categorical_accuracy: 0.68 - ETA: 16s - loss: 0.7626 - sparse_categorical_accuracy: 0.68 - ETA: 16s - loss: 0.7560 - sparse_categorical_accuracy: 0.68 - ETA: 15s - loss: 0.7518 - sparse_categorical_accuracy: 0.68 - ETA: 14s - loss: 0.7462 - sparse_categorical_accuracy: 0.68 - ETA: 14s - loss: 0.

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Assets written to: /home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/Trainer/model/11/serving_model_dir/assets


INFO:tensorflow:Assets written to: /home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/Trainer/model/11/serving_model_dir/assets
INFO:absl:Training complete. Model written to /home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/Trainer/model/11/serving_model_dir. ModelRun written to /home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/Trainer/model_run/11
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 11
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 14
        type_id: 17
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/Trainer/model/11"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 17
        name: "Model"
        )]
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 15
        type_id: 18
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/Trainer/model_run/11"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "ModelRun"
        )]
        ))

### <font color='blue'> Evaluator - Evaluating trained models </font>

- The `Evaluator` component analyzes model performance using the [TensorFlow Model Analysis library](https://www.tensorflow.org/tfx/model_analysis/get_started). 
- It runs inference requests on particular subsets of the test dataset, based on which slices are defined. 
- The `Evaluator` can also optionally validate a newly trained model against a previous model.
- Use the `ResolverNode` to pick the previous model to compare against (The model resolver is only required if performing model validation in addition to evaluation - In this case we validate against the latest blessed model. If no model has been blessed before, as in this case, the evaluator will make our candidate the first blessed model.)


In [49]:
model_resolver = ResolverNode(
      instance_name='latest_blessed_model_resolver',
      resolver_class=latest_blessed_model_resolver.LatestBlessedModelResolver,
      model=Channel(type=Model),
      model_blessing=Channel(type=ModelBlessing))

In [50]:
context.run(model_resolver)

INFO:absl:Running driver for ResolverNode.latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running publisher for ResolverNode.latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ResolverNode.latest_blessed_model_resolver
    execution_id: 12
    outputs:
        model: Channel(
            type_name: Model
            artifacts: []
        )
        model_blessing: Channel(
            type_name: ModelBlessing
            artifacts: []
        ))

#### Configure evaluation metrics and slices - create a tfma.MetricThreshold

In [51]:
accuracy_threshold = tfma.MetricThreshold(
                value_threshold=tfma.GenericValueThreshold(
                    lower_bound={'value': 0.5},
                    upper_bound={'value': 0.99})
                )


metrics_specs = tfma.MetricsSpec(
                   metrics = [
                       tfma.MetricConfig(class_name='SparseCategoricalAccuracy',
                           threshold=accuracy_threshold),
                       tfma.MetricConfig(class_name='ExampleCount')])


eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(label_key='Cover_Type')
    ],
    metrics_specs=[metrics_specs],
    slicing_specs=[
        tfma.SlicingSpec(),
        tfma.SlicingSpec(feature_keys=['Wilderness_Area'])
    ]
)


eval_config

model_specs {
  label_key: "Cover_Type"
}
slicing_specs {
}
slicing_specs {
  feature_keys: "Wilderness_Area"
}
metrics_specs {
  metrics {
    class_name: "SparseCategoricalAccuracy"
    threshold {
      value_threshold {
        lower_bound {
          value: 0.5
        }
        upper_bound {
          value: 0.99
        }
      }
    }
  }
  metrics {
    class_name: "ExampleCount"
  }
}

In [52]:
model_analyzer = Evaluator(
    examples=example_gen.outputs.examples,
    model=trainer.outputs.model,
    baseline_model=model_resolver.outputs.model,
    eval_config=eval_config
)

In [53]:
context.run(model_analyzer, enable_cache=False)

INFO:absl:Running driver for Evaluator
INFO:absl:MetadataStore with DB connection initialized
2023-02-13 20:35:00.363150: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:581] No property is defined for the Type
2023-02-13 20:35:00.369136: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:581] No property is defined for the Type
INFO:absl:Running executor for Evaluator
        please use "has_baseline" instead.
INFO:absl:Request was made to ignore the baseline ModelSpec and any change thresholds. This is likely because a baseline model was not provided: updated_config=
model_specs {
  label_key: "Cover_Type"
}
slicing_specs {
}
slicing_specs {
  feature_keys: "Wilderness_Area"
}
metrics_specs {
  metrics {
    class_name: "SparseCategoricalAccuracy"
    threshold {
      value_threshold {
        lower_bound {
          value: 0.5
        }
        upper_bound {
          value: 0.99
        }
      }
    }
  }
  metrics {
    class_name: "ExampleCount"
  }
}

IN

ExecutionResult(
    component_id: Evaluator
    execution_id: 13
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(artifact: id: 16
        type_id: 23
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/Evaluator/evaluation/13"
        custom_properties {
          key: "name"
          value {
            string_value: "evaluation"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "ModelEvaluation"
        )]
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 17
        type_id: 24
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/Evaluator/blessing/13"
        custom_properties {
          key: "blessed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "current_model"
          value {
            string_value: "/home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/Trainer/model/11"
          }
        }
        custom_properties {
          key: "current_model_id"
          value {
            int_value: 14
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 24
        name: "ModelBlessing"
        )]
        ))

### <font color='blue'> InfraValidator - validating a candidate model </font>

- The `InfraValidator` component acts as an additional early warning layer by validating a candidate model in a sandbox version of its serving infrastructure to prevent an unservable model from being pushed to production. 
- Compared to the `Evaluator` component above which validates a model's performance, the `InfraValidator` component is validating that a model is able to generate predictions from served examples in an environment configured to match production. 
- The config below takes a model and examples, launches the model in a sand-boxed [TensorflowServing](https://www.tensorflow.org/tfx/guide/serving) model server from the latest image in a local docker engine, and optionally checks that the model binary can be loaded and queried before "blessing" it for production.


In [59]:
infra_validator = InfraValidator(
    model=trainer.outputs['model'],
    examples=example_gen.outputs['examples'],
    serving_spec=infra_validator_pb2.ServingSpec(
        tensorflow_serving=infra_validator_pb2.TensorFlowServing(
            tags=['latest']),
      local_docker=infra_validator_pb2.LocalDockerConfig(),
  ),
    validation_spec=infra_validator_pb2.ValidationSpec(
        max_loading_time_seconds=60,
        num_tries=5,
    ),    
  request_spec=infra_validator_pb2.RequestSpec(
      tensorflow_serving=infra_validator_pb2.TensorFlowServingRequestSpec(),
          num_examples=5,
      )
)

In [60]:
context.run(infra_validator, enable_cache=False)

INFO:absl:Running driver for InfraValidator
INFO:absl:MetadataStore with DB connection initialized
2023-02-13 20:35:27.372741: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:581] No property is defined for the Type
INFO:absl:Running executor for InfraValidator
INFO:absl:InfraValidator will be run in LOAD_AND_QUERY mode.
INFO:absl:tag_set is not given. Using {'serve'} instead.
INFO:absl:signature_names are not given. Using ['serving_default'] instead.
INFO:absl:Creating temp directory at /home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/.temp/14/
INFO:absl:Starting infra validation (attempt 1/5).
INFO:absl:Starting LocalDockerRunner(image: tensorflow/serving:latest).
INFO:absl:Running container with parameter {'auto_remove': True, 'detach': True, 'publish_all_ports': True, 'image': 'tensorflow/serving:latest', 'environment': {'MODEL_NAME': 'infra-validation-model', 'MODEL_BASE_PATH': '/model'}, 'mounts': [{'Target': '/model/infra-validation-model/1', 'S

ExecutionResult(
    component_id: InfraValidator
    execution_id: 14
    outputs:
        blessing: Channel(
            type_name: InfraBlessing
            artifacts: [Artifact(artifact: id: 18
        type_id: 26
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/InfraValidator/blessing/14"
        custom_properties {
          key: "blessed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "InfraValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 26
        name: "InfraBlessing"
        )]
        ))

#### Check the model infrastructure validation status

In [61]:
infra_blessing_uri = infra_validator.outputs.blessing.get()[0].uri
!ls -l {infra_blessing_uri}

total 0
-rw-r--r-- 1 jupyter jupyter 0 Feb 13 20:35 INFRA_BLESSED


### <font color='blue'> Pusher - Deploying blessed models </font>
- The `Pusher` component checks whether a model has been "blessed", and if so, deploys it by pushing the model to a well known file destination.


In [62]:
# Configure and run the Pusher component
trainer.outputs['model']

Channel(
    type_name: Model
    artifacts: [Artifact(artifact: id: 14
type_id: 17
uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/Trainer/model/11"
custom_properties {
  key: "name"
  value {
    string_value: "model"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "Trainer"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
state: LIVE
, artifact_type: id: 17
name: "Model"
)]
)

In [63]:
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=model_analyzer.outputs['blessing'],
    infra_blessing=infra_validator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=SERVING_MODEL_DIR)))
context.run(pusher)

INFO:absl:Running driver for Pusher
INFO:absl:MetadataStore with DB connection initialized
2023-02-13 20:35:54.529879: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:581] No property is defined for the Type
INFO:absl:Running executor for Pusher
INFO:absl:Model version: 1676320554
INFO:absl:Model written to serving path /home/jupyter/serving_model/1676320554.
INFO:absl:Model pushed to /home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/Pusher/pushed_model/15.
INFO:absl:Running publisher for Pusher
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Pusher
    execution_id: 15
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(artifact: id: 19
        type_id: 28
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/Pusher/pushed_model/15"
        custom_properties {
          key: "name"
          value {
            string_value: "pushed_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Pusher"
          }
        }
        custom_properties {
          key: "pushed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "pushed_destination"
          value {
            string_value: "/home/jupyter/serving_model/1676320554"
          }
        }
        custom_properties {
          key: "pushed_version"
          value {
            string_value: "1676320554"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 28
        name: "PushedModel"
        )]
        ))

#### Examine the output of `Pusher`

In [64]:
pusher.outputs

{'pushed_model': Channel(
    type_name: PushedModel
    artifacts: [Artifact(artifact: id: 19
type_id: 28
uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20230213_202643/Pusher/pushed_model/15"
custom_properties {
  key: "name"
  value {
    string_value: "pushed_model"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "Pusher"
  }
}
custom_properties {
  key: "pushed"
  value {
    int_value: 1
  }
}
custom_properties {
  key: "pushed_destination"
  value {
    string_value: "/home/jupyter/serving_model/1676320554"
  }
}
custom_properties {
  key: "pushed_version"
  value {
    string_value: "1676320554"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
state: LIVE
, artifact_type: id: 28
name: "PushedModel"
)]
)}